In [ ]:
https://realpython.com/build-recommendation-engine-collaborative-filtering/
https://www.analyticsvidhya.com/blog/2018/06/comprehensive-guide-recommendation-engine-python/
https://www.datacamp.com/community/tutorials/recommender-systems-python

In [1]:
#load libraries
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, auc, recall_score, precision_score, f1_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder  
from sklearn.preprocessing import StandardScaler  
from scipy import sparse


In [2]:
#load data
train = pd.read_csv('CC2020_train_final.csv')
test = pd.read_csv('CC2020_test_final.csv')

In [4]:
train.shape

(199476, 7)

In [5]:
train.head()

,YEAR,LOCATION,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,18,Loc 5608,Inbred_142,Cluster11,Tester_740,Cluster10,1.135462
1,18,Loc 4533,Inbred_142,Cluster11,Tester_740,Cluster10,1.139813
2,18,Loc 5620,Inbred_142,Cluster11,Tester_740,Cluster10,1.117778
3,18,Loc 4732,Inbred_142,Cluster11,Tester_740,Cluster10,1.171366
4,18,Loc 5500,Inbred_142,Cluster11,Tester_740,Cluster10,1.059364


In [6]:
train.describe()

,YEAR,YIELD
count,199476.000000,199476.000000
mean,17.160551,1.001731
std,0.741779,0.104722
min,16.000000,0.047236
25%,17.000000,0.941870
50%,17.000000,1.003277
75%,18.000000,1.064073
max,18.000000,1.800083


In [7]:
test.head()

,INBRED,INBRED_CLUSTER,TESTER,TESTER_CLUSTER,YIELD
0,Inbred_1071,Cluster8,Tester_5450,Cluster5,NaN
1,Inbred_122,Cluster12,Tester_4336,Cluster6,NaN
2,Inbred_1337,Cluster17,Tester_2652,Cluster1,NaN
3,Inbred_1337,Cluster17,Tester_4373,Cluster3,NaN
4,Inbred_1339,Cluster17,Tester_4238,Cluster11,NaN


In [8]:
test.describe()

,YIELD
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [ ]:
#EDA

In [ ]:
#other libararies
#lightfm, recsys, turicreate, scipy, suprise, seaborn

In [ ]:
#metrics
#accuracy, recall, mean reciprocal rank, mean average precision at cutoff k, mean square error

In [ ]:
#data cleaning

In [ ]:
#feature engineering

In [ ]:
#modeling